In [10]:
# 실행 시간 기준 00:15
# 날짜 검색 기준 전날 00:00 ~ 24:00
import datetime

now = datetime.datetime.now()
nowDate = now.strftime('%Y-%m-%d')
print(nowDate)

2022-11-28


In [9]:
# 모든 파일 scan 하여 file_data_arr에 담아두기 dict형태
import glob

file_arr = glob.glob('../data/newsml/*/*/*/*/*.xml')
file_dict_list = []
for file in file_arr :
    file_dict = dict()
    file_dict['xml_path'] = file
    file_dict['NewsItemId'] = file.split('\\')[5].replace(".xml","")
    file_dict_list.append(file_dict)

[{'xml_path': '../data/newsml\\01100201\\2021\\11\\11\\01100201.20211111000455001.xml', 'NewsItemId': '01100201.20211111000455001'}, {'xml_path': '../data/newsml\\01100201\\2021\\11\\11\\01100201.20211111000553001.xml', 'NewsItemId': '01100201.20211111000553001'}, {'xml_path': '../data/newsml\\01100201\\2021\\11\\11\\01100201.20211111000712001.xml', 'NewsItemId': '01100201.20211111000712001'}, {'xml_path': '../data/newsml\\01100201\\2021\\11\\11\\01100201.20211111000712002.xml', 'NewsItemId': '01100201.20211111000712002'}, {'xml_path': '../data/newsml\\01100201\\2021\\11\\11\\01100201.20211111001248001.xml', 'NewsItemId': '01100201.20211111001248001'}, {'xml_path': '../data/newsml\\01400351\\2003\\09\\02\\01400351.20030902085903010.xml', 'NewsItemId': '01400351.20030902085903010'}, {'xml_path': '../data/newsml\\01400351\\2003\\09\\02\\01400351.20030902085903011.xml', 'NewsItemId': '01400351.20030902085903011'}, {'xml_path': '../data/newsml\\01400351\\2003\\09\\02\\01400351.200309020859

In [11]:
# log DB에서 불러오기

import pymysql
from elasticsearch import Elasticsearch
# MySQL Connection 연결
con = pymysql.connect(host='192.168.0.227', user='user', password='lab13579'
    ,db='NEWSDB', charset='utf8') # 한글처리 (charset = 'utf8')

# Connection 으로부터 Cursor 생성
cur = con.cursor()

# SQL문 실행 및 Fetch
sql = """
    SELECT NEWSITEMID
    FROM tb_batch_log 
    WHERE date(batch_date) BETWEEN %s AND %s
        AND status_cd = 'FAIL'
"""
cur.execute(sql, (str(nowDate), str(nowDate)))

# 데이타 Fetch
rows = cur.fetchall()

# 전날의 es insert 실패 아이디 목록
fail_id_list = []

for row in rows:
    fail_id_list.append(row[0])

print(fail_id_list)

['01100201.20211111000455001', '01400351.20030902085903010']


In [ ]:
# log DB에서 에러 발생한 NewsItemId xml읽고 추가
for fail_id in fail_id_list:
    for file_dict in file_dict_list:
        if fail_id == file_dict['NewsItemId']:
            add_doc(file_dict['xml_path'])

In [22]:
# es에서 불러오기
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9199')
# print(str(nowDate))

index = 'kpf_bigkindslab_v1.1_'+str(nowDate)[:4]
print(index)

body = {
    "query":{
        "range":{
            "@timestamp": {
                "gte": str(nowDate),
                "lte": str(nowDate)
            }
        }
    }
}

es_results = es.search(index="kpf_bigkindslab_v.1.1_202111", body=body)

for result in es_results['hits']['hits']:
    print(result)

kpf_bigkindslab_v1.1_2022


C:\Users\user\AppData\Local\Temp\ipykernel_6464\2586177558.py:21: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es_results = es.search(index="kpf_bigkindslab_v.1.1_202111", body=body)


NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [kpf_bigkindslab_v.1.1_202111]', kpf_bigkindslab_v.1.1_202111, index_or_alias)

In [ ]:
# NewsItemId 값 비교 (파일 scan, es)

# 파일 scan에 있지만 es에 등록되지 않은 NewsItemId 찾아 xml 읽어낸 후 es에 추가

In [5]:
# es에 추가하는 def
from elasticsearch import Elasticsearch
from newsml_to_json import make_json

es = Elasticsearch('http://localhost:9199')

mapping = {
    "settings" : {
        "number_of_shards": 6,
        "number_of_replicas": 1
    }
}

def add_doc(xml_path):
    # xml -> json 변환
    json_dict = make_json(xml_path)
    print(json_dict)
    index = "kpf_bigkindslab_v1.1_" + json_dict['NewsEnvelope']['DateAndTime'][:6]
    news_id = json_dict['NewsItem']['Identification']['NewsIdentifier']['NewsItemId']
    print(news_id)
    # index가 존재하는 지 확인
    # index 없을 경우 index 생성 후 doc 추가
    if es.indices.exists(index=index)==False:
        es.indices.create(index=index, body=mapping)
        # realtime_logger.info('index create : '+date)

    result = es.index(index=index, doc_type="_doc", body=json_dict, id='test') 

add_doc('../data/01100201.20211111000455001.xml')

{'Catalog': {'Href': 'http://newsml.or.kr/topicset/MasterCatalog.xml'}, 'NewsEnvelope': {'DateAndTime': '20211111T000455+0900'}, 'NewsItem': {'Identification': {'NewsIdentifier': {'ProviderId': 'kmib.co.kr', 'DateId': '20211111', 'NewsItemId': '01100201.20211111000455001', 'RevisionId': {'PreviousRevision': '1', 'Update': 'U', 'text': '2'}, 'PublicIdentifier': 'kmib.co.kr:20211111:01100201.20211111000455001:2'}}, 'NewsManagement': {'NewsItemType': {'FormalName': 'News'}, 'FirstCreated': '20211201T194214+0900', 'ThisRevisionCreated': '20211111T000355+0900', 'Status': {'FormalName': 'Usable'}, 'StatusWillChange': {'FutureStatus': {'FormalName': 'Usable'}, 'DateAndTime': '20211201T194214+0900'}, 'Urgency': {'FormalName': '6'}}, 'NewsComponent': {'Role': {'FormalName': 'Main'}, 'NewsLines': {'HeadLine': '고무장갑 손, 기름에 넣고 탕수육 튀긴 고교생 태준이', 'SubHeadLine': '<고졸 노동자, 그 서러운 이름> ③ 일자리 보장 받으려 위험 노동', 'ByLine': '박민지', 'DateLine': '20211111T000355+0900', 'CreditLine': None, 'CopyrightLine': 'Copyright

C:\Users\user\AppData\Local\Temp\ipykernel_6464\3717368670.py:27: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.index(index=index, doc_type="_doc", body=json_dict, id='test')


In [6]:
from datetime import datetime as dt
# import datetime
# print(datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z')

now = dt.now().strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
print(now)
# now.strftime('%Y%m%dT%H%M%S')

2022-11-28T17:57:13.869Z
